In [ ]:
import cv2
import matplotlib.pyplot as plt
import pyflow.pyflow as pyflow
import numpy as np
from evaluation import *

%load_ext autoreload
%autoreload 2

In [ ]:
# Code imported from https://github.com/mcv-m6-video/mcv-m6-2023-team2/blob/main/week4/utils_w4.py
def convert_optical_flow_to_image(flow):
    # The 3-channel uint16 PNG images that comprise optical flow maps contain information
    # on the u-component in the first channel, the v-component in the second channel,
    # and whether a valid ground truth optical flow value exists for a given pixel in the third channel.
    # A value of 1 in the third channel indicates the existence of a valid optical flow value
    # while a value of 0 indicates otherwise. To convert the u- and v-flow values from
    # their original uint16 format to floating point values, one can do so by subtracting 2^15 from the value,
    # converting it to float, and then dividing the result by 64.

    img_u = (flow[:, :, 2] - 2 ** 15) / 64
    img_v = (flow[:, :, 1] - 2 ** 15) / 64

    assert (flow[:, :, 0] > 1).sum() == 0 # all values are 0 or 1

    img_u[flow[:, :, 0] == 0] = 0
    img_v[flow[:, :, 0] == 0] = 0

    optical_flow = np.dstack((img_u, img_v, flow[:, :, 0]))
    return optical_flow

gt_flow = cv2.imread(r'kitti_data\training\flow_noc\000045_10.png', cv2.IMREAD_UNCHANGED).astype(np.double)
gt_flow_img = convert_optical_flow_to_image(gt_flow)

In [ ]:
first = cv2.imread(r'kitti_data\training\image_0\000045_10.png').astype(float)  / 255
second = cv2.imread(r'kitti_data\training\image_0\000045_11.png').astype(float) / 255

# alphas = [0.005, 0.01, 0.05, 0.1]
# ratios = [0.25, 0.5, 0.75, 0.9]
# minWidths = [5, 10, 30]
# nOuterFPIterations_list = [3, 7, 11]
# nInnerFPIterations_list = [1, 5, 9]
# nSORIterations_list = [30, 70]

alpha = 0.01
ratio = 0.5
minWidth = 30
nOuterFPIterations = 7
nInnerFPIterations = 1
nSORIterations = 30

u, v, im2W = pyflow.coarse2fine_flow(first, second, alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations, nSORIterations)

pred_flow = np.dstack((u, v, np.ones_like(u)))

msen, pepn = compute_errors(gt_flow_img, pred_flow, 3, 'results_pyflow/', plots=True)
opticalFlow_arrows(first, gt_flow_img, pred_flow, 'results_pyflow/', 'pyflow')
HSVOpticalFlow2(gt_flow_img, 'results_pyflow/', 'pyflow_gt')
HSVOpticalFlow2(pred_flow, 'results_pyflow/', 'pyflow_pred')
print('MSEN:', msen)
print('PEPN:', pepn)

In [ ]:
first = cv2.imread(r'kitti_data\training\image_0\000045_10.png')
second = cv2.imread(r'kitti_data\training\image_0\000045_11.png')
first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
second_gray = cv2.cvtColor(second, cv2.COLOR_BGR2GRAY)
# Assuming prev_gray is your previous grayscale image
h, w = first_gray.shape

# Create a grid of points
y, x = np.mgrid[0:h, 0:w].astype(np.float32)
points = np.stack((x, y), axis=-1).reshape(-1, 1, 2)

new_points, status, error = cv2.calcOpticalFlowPyrLK(first_gray, second_gray, points, None, maxLevel=3, winSize=(45,45))

# Create a mask to select good points
new_points[status == 0] = 0
points[status == 0] = 0

flow = new_points - points

u = flow[:, 0, 0].reshape(h, w)
v = flow[:, 0, 1].reshape(h, w)

pred_flow = np.stack((u, v, np.ones_like(u)), axis=2)

msen, pepn = compute_errors(gt_flow_img, pred_flow, 3, 'results_kl/', plots=True)
opticalFlow_arrows(first, gt_flow_img, pred_flow, 'results_kl/', 'kl')
HSVOpticalFlow2(pred_flow, 'results_kl/', 'kl_pred')
print('MSEN:', msen)
print('PEPN:', pepn)

In [6]:
from core.raft import RAFT
from core.utils.utils import InputPadder
import torch

def load_image(img):
    img = np.array(img).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None]

import argparse

# Define a function to create and parse arguments
def create_args(small):
    parser = argparse.ArgumentParser()
    parser.add_argument('--small', action='store_true', help='use small model', default=small)
    parser.add_argument('--mixed_precision', action='store_true', help='use mixed precision')
    args, unknown = parser.parse_known_args()  # This line parses the arguments
    return args

# Create an 'args' variable with default values or specified values.
args = create_args(small=False)

first = cv2.imread(r'kitti_data\training\image_0\000045_10.png')
second = cv2.imread(r'kitti_data\training\image_0\000045_11.png')

shape = first.shape

image1 = load_image(first)
image2 = load_image(second)

model = torch.nn.DataParallel(RAFT(args))
model.load_state_dict(torch.load('raft-kitti.pth', map_location=torch.device('cpu')))
model.eval()

padder = InputPadder(image1.shape)
image1, image2 = padder.pad(image1, image2)

with torch.no_grad():
    flow_low, flow_up = model(image1, image2, iters=20, test_mode=True)

# Resize flow_up to original image1 size
flow_up = torch.nn.functional.interpolate(flow_up, size=shape[:2], mode='bilinear', align_corners=True)
flow_up = flow_up.numpy().squeeze().transpose(1, 2, 0)

pred_flow = np.stack((flow_up[:, :, 0], flow_up[:, :, 1], np.ones_like(flow_up[:, :, 0])), axis=2)
msen, pepn = compute_errors(gt_flow_img, pred_flow, 3, 'results_raft/', plots=True)
opticalFlow_arrows(first, gt_flow_img, pred_flow, 'results_raft/', 'raft')
HSVOpticalFlow2(pred_flow, 'results_raft/', 'raft_pred')
print('MSEN:', msen)
print('PEPN:', pepn)